In [ ]:
data_path = '../dataset'

In [ ]:
import os

In [ ]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

In [ ]:
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')

In [ ]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

In [ ]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

In [7]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [9]:
torch.cuda.empty_cache()

In [10]:
from torchvision import transforms as T

In [11]:
train_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((224,224)),
    T.RandomVerticalFlip(.5),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
val_transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
from torchvision.datasets import ImageFolder

In [14]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=train_transform
)

In [15]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 750
    Root location: ../dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               RandomVerticalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [16]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=val_transform
)

In [17]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2500
    Root location: ../dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [18]:
from torch.utils.data import DataLoader

In [19]:
BATCH_SIZE = 16

In [20]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [21]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=False
)

In [22]:
import torch.nn as nn
import torch.nn.functional as F

In [23]:
class FoodNet(nn.Module):
    def __init__(self):
        super(FoodNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 8, 3)
        self.conv3 = nn.Conv2d(8, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(.1)
        self.fc1 = nn.Linear(16 * 26 * 26, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 16 * 26 * 26)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = self.fc3(x)
        return x

In [24]:
model = FoodNet()
model = model.to(device)

In [25]:
import torch.optim as optim

In [26]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [27]:
loaders = {
    'train': train_loader,
    'val': test_loader
}

In [28]:
from copy import deepcopy

In [29]:
%%time

EPOCHS = 20

for epoch in range(1, EPOCHS+1):
    running_loss = .0
    running_acc = .0
    best_acc = .0
    best_model = None
    print(f"\nEpoch {epoch}/{EPOCHS}\n{'='*25}")
    for phase in ['train', 'val']:
        if phase == 'train': model.train()
        if phase == 'val': model.eval()
        for inputs, targets in loaders[phase]:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            accuracy = (torch.argmax(outputs, dim=1) == targets).float().mean()

            running_loss += loss.item()
            running_acc += accuracy.item()
        if phase == 'val' and accuracy > best_acc:
            best_acc = accuracy
            best_model = deepcopy(model.state_dict())
        print(f"Loss ({phase}): {running_loss/len(loaders[phase])}, Acc ({phase}): {running_acc/len(loaders[phase])}")


Epoch 1/20
Loss (train): 2.3074508575682944, Acc (train): 0.1253799393456033
Loss (val): 2.991362560326886, Acc (val): 0.14939717929454366

Epoch 2/20
Loss (train): 2.2710729254052993, Acc (train): 0.1491261398855676
Loss (val): 2.9175783486882594, Acc (val): 0.21223839856446927

Epoch 3/20
Loss (train): 2.2162357847741307, Acc (train): 0.15862462019666712
Loss (val): 2.9369459972260104, Acc (val): 0.19597361241556277

Epoch 4/20
Loss (train): 2.156119283209456, Acc (train): 0.18123100317539054
Loss (val): 2.8122154557780856, Acc (val): 0.2501137398040978

Epoch 5/20
Loss (train): 2.115331807035081, Acc (train): 0.21865501555990666
Loss (val): 2.805284680074947, Acc (val): 0.23384895370264722

Epoch 6/20
Loss (train): 2.090958889494551, Acc (train): 0.22416413401035554
Loss (val): 2.8416542763922625, Acc (val): 0.27053002737889625

Epoch 7/20
Loss (train): 2.0642273324601192, Acc (train): 0.24164133756718736
Loss (val): 2.743940590293544, Acc (val): 0.3095996360869924

Epoch 8/20
Loss

In [30]:
torch.save(best_model, 'foodnet.pth')